CHATBOT FTMM

In [6]:
import json
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.memory import ConversationBufferWindowMemory
import os
from tqdm import tqdm
import chromadb

In [2]:
loader = DirectoryLoader("knowledges")
docs = loader.load()

In [21]:
client = chromadb.PersistentClient(path="database")
collection_ftmm = client.get_or_create_collection(name="ftmm")

In [4]:
# client.reset()

True

In [7]:
# Split into chunks
text_splitter = TokenTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200
)

chunks = text_splitter.split_documents(docs)

In [17]:
metadata = [chunks[i].metadata for i in range(len(chunks))]
documents = [chunks[i].page_content for i in range(len(chunks))]
ids = [f"doc{i}" for i in range(len(chunks))]

In [18]:
embed = OpenAIEmbeddings()
embeddings = embed.embed_documents(documents)

In [25]:
status = collection_ftmm.add(
    ids=ids,
    documents=[c.page_content for c in chunks],
    embeddings=embeddings,
    metadatas=metadata
)

In [26]:
docsearch = Chroma(persist_directory="database", embedding_function=embed, collection_name="ftmm")
docsearch._collection.count()

218

In [135]:
# LLM
chat_model = ChatOpenAI()

# Template prompt
template = """Jawablah pertanyaan di bawah ini berdasarkan konteks yang diberikan! \
Jika dalam pertanyaan merujuk ke histori chat sebelumnya, maka gunakan konteks dari pertanyaan \
sebelumnya untuk menjawab!
Konteks:
{context}

Pertanyaan:
{question}
"""

template_system = """Namamu adalah Iris, sebuah chatbot Fakultas Teknologi Maju dan \
Multidisiplin (FTMM), Universitas Airlangga. Kamu siap menjawab pertanyaan apapun \
seputar FTMM. Kamu menjawab setiap pertanyaan dengan ceria, sopan, dan asik!
"""

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(template_system),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(template),
    ]
)

# Memory
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=1)

In [136]:
chain = prompt | chat_model

In [143]:
question = "ceritakan sejarah FTMM!"
contexts = docsearch.similarity_search(question, k=3)
response = chain.invoke({"context": contexts, "question": question, "chat_history": memory.buffer_as_messages})

# Add response to memory
memory.save_context({"input": template.format(context=contexts, question=question)}, 
                    {"output": response.content})
response

AIMessage(content='Fakultas Teknologi Maju dan Multidisiplin (FTMM) didirikan pada bulan November 2019 dengan nama awal "Sekolah Teknologi Maju dan Multidisiplin". FTMM hadir sebagai pelengkap dari tiga pilar utama Universitas Airlangga yaitu ilmu sosial, ilmu kehidupan, dan ilmu kesehatan, dengan menambahkan pilar baru di bidang keteknikan.\n\nFTMM memiliki lima program studi S1 yang terdiri dari Teknik Robotika dan Kecerdasan Buatan, Teknologi Sains Data, Rekayasa Nanoteknologi, Teknik Industri, dan Teknik Elektro. Pada tahun pertama, jumlah mahasiswa FTMM sekitar 400 orang.\n\nSebagai fakultas baru, FT')

In [12]:
from langchain.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
import streamlit as st
genai.configure(api_key=st.secrets["google_api_key"])

def embed_fn(text):
    embeddings = genai.embed_content(model="models/embedding-001",
                                    content=text,
                                    task_type="retrieval_document")["embedding"]
    return embeddings
docsearch = Chroma(persist_directory="knowledge_ftmm", collection_name="gemini_ftmm")

In [6]:
text = "Halo"
embeddings = embed_fn(text)

In [13]:
docsearch.similarity_search_by_vector(embeddings)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(page_content='Identitas Fakultas Teknologi Maju dan Multidisiplin Bendera Warna biru \xa0prusia \xa0(prussian \xa0blue) \xa0melambangkan\xa0 FTMM \xa0sebagai \xa0pilar \xa0teknologi \xa0di Universitas Airlangga.Warna perak (silver) memiliki makna kemajuan dan harapan sesuai dengan filosofi didirikannya FTMM sebagai bentuk kontribusi untuk Indonesia yang lebih maju melalui penguasaan teknologi.Warna \xa0pompadour\xa0 \xa0melambangkan\xa0 \xa0harmonisasi\xa0\xa0 dan\xa0 \xa0kolaborasi\xa0\xa0 multidisiplin\xa0 \xa0ilmu sebagaimana visi pendirian FTMM yang turut berperan aktif dalam pengembangan, penerapan dan kolaborasi IPTEK yang maju.Tiga buah warna yang berjajar melambangkan FTMM yang berangkat dari 3 pilar yang telah ada (health science, life science dan social science), teknologi hadir sebagai katalis kolaborasi antar pilar sehingga membawa keseimbangan bagi Universitas Airlangga sebagai institusi yang komprehensif. Logo Bentuk huruf tanpa sudut melambangkan FTMM sebagai F